In [2]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # Use non-interactive Agg backend for saving plots
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, GRU, Input, Dropout, Conv1D, MaxPooling1D, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import wfdb
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load MIT-BIH Arrhythmia Database (record 100, first 60,000 samples)
record_path = 'mit-bih-arrhythmia-database-1.0.0/100'
try:
    record = wfdb.rdrecord(record_path)
    ecg_signal = record.p_signal[:60000, 0]  # Use first 60,000 samples, MLII lead
    fs = record.fs  # Sampling frequency (360 Hz)
except Exception as e:
    print(f"Error loading record {record_path}: {e}")
    raise

# Convert to NumPy array and check for invalid values
ecg_signal = np.asarray(ecg_signal, dtype=np.float32)
if np.any(np.isnan(ecg_signal)) or np.any(np.isinf(ecg_signal)):
    print("Warning: ECG signal contains NaN or inf values. Replacing with zeros.")
    ecg_signal = np.nan_to_num(ecg_signal, nan=0.0, posinf=0.0, neginf=0.0)

print(f"ECG record loaded: {record_path}")
print(f"Sampling frequency: {fs} Hz")
print(f"Total samples: {len(ecg_signal)}")
print(f"Duration: {len(ecg_signal) / fs:.2f} seconds")
print(f"ECG signal shape: {ecg_signal.shape}, dtype: {ecg_signal.dtype}")

# Create time array
time = np.arange(len(ecg_signal)) / fs
print(f"Time array shape: {time.shape}, dtype: {time.dtype}")

# Plot raw ECG signal (first 10 seconds)
plt.figure(figsize=(14, 7))
plt.plot(time[:int(10 * fs)], ecg_signal[:int(10 * fs)])
plt.title('Raw ECG Signal (Record 100, MLII Lead, First 10 Seconds)')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude (mV)')
plt.grid(True)
plt.tight_layout()
plt.savefig('ecg_raw_signal.png')
plt.close()

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
ecg_scaled = scaler.fit_transform(ecg_signal.reshape(-1, 1))

# Plot a sample sequence
sequence_length = 180  # ~0.5 seconds at 360 Hz
horizon = 60  # Predict 60 sample ahead
sample_idx = 1000
sample_seq = ecg_scaled[sample_idx:sample_idx + sequence_length]
sample_target = ecg_scaled[sample_idx + sequence_length:sample_idx + sequence_length + horizon]
sample_time = np.arange(sequence_length + horizon) / fs
plt.figure(figsize=(14, 7))
plt.plot(sample_time[:sequence_length], sample_seq, label='Input Sequence (180 samples)')
plt.plot(sample_time[sequence_length:], sample_target, 'ro', label='Target (1 sample)')
plt.title('Sample ECG Sequence for Training')
plt.xlabel('Time (seconds)')
plt.ylabel('Normalized Amplitude')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('ecg_sample_sequence.png')
plt.close()

# Function to create sequences for time series prediction
def create_sequences(data, seq_length, horizon=1):
    X, y = [], []
    for i in range(len(data) - seq_length - horizon + 1):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length:i+seq_length+horizon])
    return np.array(X), np.array(y)

# Parameters
test_split = 0.2  # Use 20% of data for testing

# Prepare the data
X, y = create_sequences(ecg_scaled, sequence_length, horizon)
split_idx = int(len(X) * (1 - test_split))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# ----------------------------------------------------------------
# Base Models Implementation
# ----------------------------------------------------------------

# 1. LSTM model
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(horizon)
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    return model

# 2. GRU model
def build_gru_model(input_shape):
    model = Sequential([
        GRU(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        GRU(50),
        Dropout(0.2),
        Dense(horizon)
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    return model

# 3. CNN model
def build_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(50, activation='relu'),
        Dense(horizon)
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    return model

# 4. MLP model
def build_mlp_model(input_shape):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(100, activation='relu'),
        Dropout(0.2),
        Dense(50, activation='relu'),
        Dense(horizon)
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    return model

# 5. Simple Autoregressive model
def ar_forecast(X):
    return X[:, -horizon:, :]

# ----------------------------------------------------------------
# Train Base Models
# ----------------------------------------------------------------

input_shape = (sequence_length, 1)

# Train LSTM
lstm_model = build_lstm_model(input_shape)
lstm_history = lstm_model.fit(X_train, y_train.reshape(y_train.shape[0], horizon),
                              epochs=10, batch_size=32, validation_split=0.1, verbose=1)
lstm_preds = lstm_model.predict(X_test)
lstm_preds = lstm_preds.reshape(lstm_preds.shape[0], horizon, 1)

# Train GRU
gru_model = build_gru_model(input_shape)
gru_history = gru_model.fit(X_train, y_train.reshape(y_train.shape[0], horizon),
                            epochs=10, batch_size=32, validation_split=0.1, verbose=1)
gru_preds = gru_model.predict(X_test)
gru_preds = gru_preds.reshape(gru_preds.shape[0], horizon, 1)

# Train CNN
cnn_model = build_cnn_model(input_shape)
cnn_history = cnn_model.fit(X_train, y_train.reshape(y_train.shape[0], horizon),
                           epochs=10, batch_size=32, validation_split=0.1, verbose=1)
cnn_preds = cnn_model.predict(X_test)
cnn_preds = cnn_preds.reshape(cnn_preds.shape[0], horizon, 1)

# Train MLP
mlp_model = build_mlp_model(input_shape)
mlp_history = mlp_model.fit(X_train, y_train.reshape(y_train.shape[0], horizon),
                           epochs=10, batch_size=32, validation_split=0.1, verbose=1)
mlp_preds = mlp_model.predict(X_test)
mlp_preds = mlp_preds.reshape(mlp_preds.shape[0], horizon, 1)

# AR forecast
ar_preds = ar_forecast(X_test)

# Verify predictions shapes
print(f"LSTM predictions shape: {lstm_preds.shape}")
print(f"GRU predictions shape: {gru_preds.shape}")
print(f"CNN predictions shape: {cnn_preds.shape}")
print(f"MLP predictions shape: {mlp_preds.shape}")
print(f"AR predictions shape: {ar_preds.shape}")
print(f"Target shape: {y_test.shape}")

# Evaluate base models
base_models = ['LSTM', 'GRU', 'CNN', 'MLP', 'AR']
predictions = [lstm_preds, gru_preds, cnn_preds, mlp_preds, ar_preds]

for name, pred in zip(base_models, predictions):
    mae = mean_absolute_error(y_test.reshape(-1, 1), pred.reshape(-1, 1))
    rmse = np.sqrt(mean_squared_error(y_test.reshape(-1, 1), pred.reshape(-1, 1)))
    print(f"{name} - MAE: {mae:.4f}, RMSE: {rmse:.4f}")

# ----------------------------------------------------------------
# RLMC Implementation (Actor-Critic with DDPG)
# ----------------------------------------------------------------

def create_state_representation(x_seq, model_histories):
    if len(model_histories) > 0:
        model_features = np.array(model_histories[-1])
        if np.std(model_features) > 0:
            model_features = (model_features - np.mean(model_features)) / np.std(model_features)
    else:
        model_features = np.zeros(len(base_models))
    time_features = x_seq.flatten()
    state = np.concatenate([time_features, model_features])
    return state.reshape(1, -1)

# Actor Network
def build_actor_network(state_dim, action_dim):
    inputs = Input(shape=(state_dim,))
    x = Dense(256, activation='relu')(inputs)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(action_dim, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Critic Network
def build_critic_network(state_dim, action_dim):
    state_input = Input(shape=(state_dim,))
    state_out = Dense(256, activation='relu')(state_input)
    state_out = Dense(128, activation='relu')(state_out)
    action_input = Input(shape=(action_dim,))
    action_out = Dense(128, activation='relu')(action_input)
    concat = Concatenate()([state_out, action_out])
    x = Dense(64, activation='relu')(concat)
    outputs = Dense(1, activation='linear')(x)
    model = Model([state_input, action_input], outputs)
    return model

# DDPG Agent for RLMC
class RLMCAgent:
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.memory = []
        self.max_memory_size = 1000
        self.batch_size = 64
        self.gamma = 0.95
        self.tau = 0.005
        self.actor = build_actor_network(state_dim, action_dim)
        self.actor_target = build_actor_network(state_dim, action_dim)
        self.actor_target.set_weights(self.actor.get_weights())
        self.actor_optimizer = Adam(learning_rate=0.001)
        self.critic = build_critic_network(state_dim, action_dim)
        self.critic_target = build_critic_network(state_dim, action_dim)
        self.critic_target.set_weights(self.critic.get_weights())
        self.critic_optimizer = Adam(learning_rate=0.002)

    def get_action(self, state, explore=True):
        action = self.actor.predict(state, verbose=0)[0]
        if explore:
            noise = np.random.normal(0, 0.1, size=self.action_dim)
            action = action + noise
            action = np.maximum(action, 0)
            action_sum = np.sum(action)
            if action_sum > 0:
                action = action / action_sum
        return action

    def remember(self, state, action, reward, next_state):
        self.memory.append((state, action, reward, next_state))
        if len(self.memory) > self.max_memory_size:
            self.memory.pop(0)

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        indices = np.random.choice(len(self.memory), self.batch_size, replace=False)
        states, actions, rewards, next_states = [], [], [], []
        for i in indices:
            states.append(self.memory[i][0][0])
            actions.append(self.memory[i][1])
            rewards.append(self.memory[i][2])
            next_states.append(self.memory[i][3][0])
        states = tf.convert_to_tensor(np.array(states), dtype=tf.float32)
        actions = tf.convert_to_tensor(np.array(actions), dtype=tf.float32)
        rewards = tf.convert_to_tensor(np.array(rewards).reshape(-1, 1), dtype=tf.float32)
        next_states = tf.convert_to_tensor(np.array(next_states), dtype=tf.float32)
        with tf.GradientTape() as tape:
            target_actions = self.actor_target(next_states)
            target_q_values = self.critic_target([next_states, target_actions])
            target_q = rewards + self.gamma * target_q_values
            current_q = self.critic([states, actions])
            critic_loss = tf.reduce_mean(tf.square(target_q - current_q))
        critic_grads = tape.gradient(critic_loss, self.critic.trainable_variables)
        if None in critic_grads or any(tf.reduce_any(tf.math.is_nan(g)) for g in critic_grads if g is not None):
            print("Warning: Some critic gradients are None or contain NaN. Skipping critic update.")
        else:
            self.critic_optimizer.apply_gradients(zip(critic_grads, self.critic.trainable_variables))
        with tf.GradientTape() as tape:
            actions_pred = self.actor(states)
            actor_loss = -tf.reduce_mean(self.critic([states, actions_pred]))
        actor_grads = tape.gradient(actor_loss, self.actor.trainable_variables)
        if None in actor_grads or any(tf.reduce_any(tf.math.is_nan(g)) for g in actor_grads if g is not None):
            print("Warning: Some actor gradients are None or contain NaN. Skipping actor update.")
        else:
            self.actor_optimizer.apply_gradients(zip(actor_grads, self.actor.trainable_variables))
        self._update_target_networks()

    def _update_target_networks(self):
        actor_weights = self.actor.get_weights()
        actor_target_weights = self.actor_target.get_weights()
        for i in range(len(actor_weights)):
            actor_target_weights[i] = self.tau * actor_weights[i] + (1 - self.tau) * actor_target_weights[i]
        self.actor_target.set_weights(actor_target_weights)
        critic_weights = self.critic.get_weights()
        critic_target_weights = self.critic_target.get_weights()
        for i in range(len(critic_weights)):
            critic_target_weights[i] = self.tau * critic_weights[i] + (1 - self.tau) * critic_target_weights[i]
        self.critic_target.set_weights(critic_target_weights)

# Reward function
def compute_reward(predicted, actual, base_predictions):
    predicted = predicted.reshape(-1)
    actual = actual.reshape(-1)
    base_predictions = [p.reshape(-1) for p in base_predictions]
    epsilon = 1e-10
    error = np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual) + epsilon)
    smape = 200 * error.mean()
    base_errors = []
    for pred in base_predictions:
        err = np.abs(pred - actual) / (np.abs(pred) + np.abs(actual) + epsilon)
        base_errors.append(200 * err.mean())
    rank = sum(1 for e in base_errors if e < smape)
    rank_normalized = 1 - 2 * (rank / len(base_models))
    error_quantile = np.searchsorted(sorted(base_errors + [smape]), smape) / 10
    error_normalized = 1 - 2 * (error_quantile / 9)
    alpha = 0.5
    reward = alpha * error_normalized + rank_normalized
    return reward

# ----------------------------------------------------------------
# Train the RLMC model
# ----------------------------------------------------------------

val_split = int(len(X_test) * 0.5)
X_val, X_rlmc_test = X_test[:val_split], X_test[val_split:]
y_val, y_rlmc_test = y_test[:val_split], y_test[val_split:]

print(f"Validation set shape: X_val {X_val.shape}, y_val {y_val.shape}")
print(f"RLMC test set shape: X_rlmc_test {X_rlmc_test.shape}, y_rlmc_test {y_rlmc_test.shape}")

val_base_preds = [
    lstm_model.predict(X_val, verbose=0).reshape(-1, horizon, 1),
    gru_model.predict(X_val, verbose=0).reshape(-1, horizon, 1),
    cnn_model.predict(X_val, verbose=0).reshape(-1, horizon, 1),
    mlp_model.predict(X_val, verbose=0).reshape(-1, horizon, 1),
    ar_forecast(X_val)
]

for i, name in enumerate(base_models):
    print(f"{name} val predictions shape: {val_base_preds[i].shape}")

state_dim = sequence_length + len(base_models)
action_dim = len(base_models)
rlmc_agent = RLMCAgent(state_dim, action_dim)

n_episodes = 15
model_histories = [np.zeros(len(base_models))]

for episode in range(n_episodes):
    total_reward = 0
    for t in range(len(X_val) - 1):
        state = create_state_representation(X_val[t], model_histories)
        action = rlmc_agent.get_action(state)
        ensemble_pred = np.zeros_like(y_val[t])
        for i, weight in enumerate(action):
            ensemble_pred += weight * val_base_preds[i][t]
        reward = compute_reward(ensemble_pred, y_val[t], [pred[t] for pred in val_base_preds])
        total_reward += reward
        current_errors = []
        for pred in val_base_preds:
            err = np.abs(pred[t] - y_val[t]).mean()
            current_errors.append(err)
        if len(model_histories) >= 5:
            model_histories.pop(0)
        model_histories.append(current_errors)
        next_state = create_state_representation(X_val[t+1], model_histories)
        rlmc_agent.remember(state, action, reward, next_state)
        rlmc_agent.replay()
    print(f"Episode {episode+1}/{n_episodes}, Average Reward: {total_reward/len(X_val):.4f}")

# ----------------------------------------------------------------
# Evaluate RLMC on test set
# ----------------------------------------------------------------

test_base_preds = [
    lstm_model.predict(X_rlmc_test, verbose=0).reshape(-1, horizon, 1),
    gru_model.predict(X_rlmc_test, verbose=0).reshape(-1, horizon, 1),
    cnn_model.predict(X_rlmc_test, verbose=0).reshape(-1, horizon, 1),
    mlp_model.predict(X_rlmc_test, verbose=0).reshape(-1, horizon, 1),
    ar_forecast(X_rlmc_test)
]

rlmc_preds = np.zeros_like(y_rlmc_test)
model_weights_history = []
model_histories = [np.zeros(len(base_models))]

for t in range(len(X_rlmc_test)):
    if t > 0:
        current_errors = []
        for pred in test_base_preds:
            err = np.abs(pred[t-1] - y_rlmc_test[t-1]).mean()
            current_errors.append(err)
        if len(model_histories) >= 5:
            model_histories.pop(0)
        model_histories.append(current_errors)
    state = create_state_representation(X_rlmc_test[t], model_histories)
    action = rlmc_agent.get_action(state, explore=False)
    model_weights_history.append(action)
    for i, weight in enumerate(action):
        rlmc_preds[t] += weight * test_base_preds[i][t]

y_rlmc_test_flat = y_rlmc_test.reshape(-1, 1)
rlmc_preds_flat = rlmc_preds.reshape(-1, 1)
rlmc_preds_rescaled = scaler.inverse_transform(rlmc_preds_flat)
y_test_rescaled = scaler.inverse_transform(y_rlmc_test_flat)

base_preds_rescaled = []
for pred in test_base_preds:
    base_preds_rescaled.append(scaler.inverse_transform(pred.reshape(-1, 1)))

rlmc_mae = mean_absolute_error(y_test_rescaled, rlmc_preds_rescaled)
rlmc_rmse = np.sqrt(mean_squared_error(y_test_rescaled, rlmc_preds_rescaled))

print("\nFinal Evaluation Results:")
print(f"RLMC - MAE: {rlmc_mae:.4f}, RMSE: {rlmc_rmse:.4f}")

# Plot the results (first 1000 samples)
time_steps = np.arange(len(y_test_rescaled)) / fs
plt.figure(figsize=(14, 7))
plt.plot(time_steps[:1000], y_test_rescaled[:1000], label='Actual ECG Signal')
plt.plot(time_steps[:1000], rlmc_preds_rescaled[:1000], label='RLMC Prediction')
for i, (name, pred) in enumerate(zip(base_models, base_preds_rescaled)):
    if i < 3:
        plt.plot(time_steps[:1000], pred[:1000], label=f'{name} Prediction', alpha=0.5)
plt.title('ECG Signal Prediction with RLMC')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude (mV)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('ecg_prediction.png')
plt.close()

# Plot model weights over time
model_weights = np.array(model_weights_history)
plt.figure(figsize=(14, 7))
for i, name in enumerate(base_models):
    plt.plot(model_weights[:, i], label=f'{name} Weight')
plt.title('RLMC Model Weights Over Time')
plt.xlabel('Time Steps')
plt.ylabel('Weight')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('model_weights.png')
plt.close()

# Performance comparison
print("\nComparison of All Models:")
base_maes = []
base_rmses = []

for i, (name, pred) in enumerate(zip(base_models, base_preds_rescaled)):
    mae = mean_absolute_error(y_test_rescaled, pred)
    rmse = np.sqrt(mean_squared_error(y_test_rescaled, pred))
    base_maes.append(mae)
    base_rmses.append(rmse)
    print(f"{name} - MAE: {mae:.4f}, RMSE: {rmse:.4f}")

avg_ensemble_pred = np.mean(base_preds_rescaled, axis=0)
avg_mae = mean_absolute_error(y_test_rescaled, avg_ensemble_pred)
avg_rmse = np.sqrt(mean_squared_error(y_test_rescaled, avg_ensemble_pred))
print(f"Average Ensemble - MAE: {avg_mae:.4f}, RMSE: {avg_rmse:.4f}")
print(f"RLMC - MAE: {rlmc_mae:.4f}, RMSE: {rlmc_rmse:.4f}")

best_base_mae = min(base_maes)
improvement_over_best = ((best_base_mae - rlmc_mae) / best_base_mae) * 100
improvement_over_avg = ((avg_mae - rlmc_mae) / avg_mae) * 100

print(f"\nRLMC improvement over best base model: {improvement_over_best:.2f}%")
print(f"RLMC improvement over average ensemble: {improvement_over_avg:.2f}%")

ECG record loaded: mit-bih-arrhythmia-database-1.0.0/100
Sampling frequency: 360 Hz
Total samples: 60000
Duration: 166.67 seconds
ECG signal shape: (60000,), dtype: float32
Time array shape: (60000,), dtype: float64
Training samples: 47808
Testing samples: 11953
X_train shape: (47808, 180, 1), y_train shape: (47808, 60, 1)
X_test shape: (11953, 180, 1), y_test shape: (11953, 60, 1)


I0000 00:00:1746262843.348395 3464133 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1210 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/10


2025-05-03 14:30:44.498828: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30979440 exceeds 10% of free system memory.
2025-05-03 14:30:44.548223: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30979440 exceeds 10% of free system memory.
I0000 00:00:1746262845.986198 3464328 cuda_dnn.cc:529] Loaded cuDNN version 90501


1345/1345 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - loss: 0.0120 - val_loss: 0.0058
Epoch 2/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - loss: 0.0056 - val_loss: 0.0040
Epoch 3/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.0041 - val_loss: 0.0033
Epoch 4/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.0035 - val_loss: 0.0032
Epoch 5/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.0032 - val_loss: 0.0035
Epoch 6/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.0031 - val_loss: 0.0037
Epoch 7/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.0030 - val_loss: 0.0035
Epoch 8/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.0029 - val_loss: 0.0031
Epoch 9/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.0028 - val_loss: 0.0035
Epoch 10/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.0028 - val_loss: 0.0030
374/374 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Epoch 1/10


...

  72/1345 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0240

I0000 00:00:1746263152.822451 3464324 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1345/1345 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 0.0100 - val_loss: 0.0062
Epoch 2/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0056 - val_loss: 0.0044
Epoch 3/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0043 - val_loss: 0.0037
Epoch 4/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0037 - val_loss: 0.0034
Epoch 5/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 6/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0032 - val_loss: 0.0030
Epoch 7/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0030 - val_loss: 0.0030
Epoch 8/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0030 - val_loss: 0.0030
Epoch 9/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0029 - val_loss: 0.0030
Epoch 10/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0028 - val_loss: 0.0028
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Epoch 1/10


2025-05-03 14:36:28.630859: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30979440 exceeds 10% of free system memory.


1345/1345 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0121 - val_loss: 0.0067
Epoch 2/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0066 - val_loss: 0.0055
Epoch 3/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0057 - val_loss: 0.0050
Epoch 4/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0052 - val_loss: 0.0044
Epoch 5/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0048 - val_loss: 0.0042
Epoch 6/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0045 - val_loss: 0.0042
Epoch 7/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0044 - val_loss: 0.0036
Epoch 8/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0043 - val_loss: 0.0037
Epoch 9/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0042 - val_loss: 0.0042
Epoch 10/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0041 - val_loss: 0.0037
374/374 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
LSTM predictions shape: (11953, 60, 1)
GRU predictions shape: (11953, 60, 1)
CNN predicti